In this notebook, we run an optimization making use of all of the functionality of horsetail matching. 

In [3]:
import numpy as np

from horsetailmatching import HorsetailMatching, UncertainParameter
from horsetailmatching.demoproblems import TP2
from horsetailmatching.surrogates import PolySurrogate

def fQOI(x, u):
    factor = np.sqrt(u[0]**2 + u[1]**2)
    q = 1 + (1 + factor*(x[0]**2 + x[1]*x[0] + x[1]**2))
    grad = [factor*(2*x[0] + x[1]), factor*(x[0] + 2*x[1])]
    return q, grad

u_1 = UncertainParameter('uniform', lower_bound=-1, upper_bound=1)
u_2 = UncertainParameter('gaussian', mean=0, standard_deviation=1)
u_3 = UncertainParameter('interval', lower_bound=-1, upper_bound=1)
             
def ftarget_u(h):
    return 0 - h**5

def ftarget_l(h):
    return 1 - h**5
             
qPolyChaos = PolySurrogate(dimensions=3, order=3, poly_type=['legendre', 'hermite', 'legendre'])
gradPolyChaos = [PolySurrogate(dimensions=3, order=3, poly_type=['legendre', 'hermite', 'legendre']),
                 PolySurrogate(dimensions=3, order=3, poly_type=['legendre', 'hermite', 'legendre'])]
u_quad_points = qPolyChaos.getQuadraturePoints()
             
def mySurrogateWithGrad(u_quad, q_quad, grad_quad):
    qPolyChaos.train(q_quad)
    for i, gPC in enumerate(gradPolyChaos):
        gPC.train(grad_quad[:, i])
    def qmodel(u):
        return qPolyChaos.predict(u)
    def gradmodel(u):
        return [gPC.predict(u) for gPC in gradPolyChaos]
    return qmodel, gradmodel

theHM = HorsetailMatching(fQOI, [u_1, u_2], jac=True, 
                          ftarg_u=ftarget_u, ftarg_l=ftarget_l,
                          n_samples_prob=1000, n_samples_int=50, q_integration_points=np.linspace(0, 10, 500),
                          surrogate=mySurrogateWithGrad, u_quadrature_points=u_quad_points, surrogate_jac=True)

theHM.evalMetric([0, 1])

TypeError: unsupported operand type(s) for ** or pow(): 'list' and 'int'